<h1 align="center" style="line-height:200%; font-family: Vazir; color: #80a8d2;">Sleep stage classification using EEG data.
</h1>



# Access to data

In [ ]:
!unzip /content/checkpoint-600.zip -d /content/checkpoint-600

Archive:  /content/checkpoint-600.zip
   creating: /content/checkpoint-600/kaggle/working/peft-EEG-training-1735478979/checkpoint-600/
  inflating: /content/checkpoint-600/kaggle/working/peft-EEG-training-1735478979/checkpoint-600/adapter_config.json  
  inflating: /content/checkpoint-600/kaggle/working/peft-EEG-training-1735478979/checkpoint-600/training_args.bin  
  inflating: /content/checkpoint-600/kaggle/working/peft-EEG-training-1735478979/checkpoint-600/rng_state.pth  
  inflating: /content/checkpoint-600/kaggle/working/peft-EEG-training-1735478979/checkpoint-600/optimizer.pt  
  inflating: /content/checkpoint-600/kaggle/working/peft-EEG-training-1735478979/checkpoint-600/adapter_model.safetensors  
  inflating: /content/checkpoint-600/kaggle/working/peft-EEG-training-1735478979/checkpoint-600/README.md  
  inflating: /content/checkpoint-600/kaggle/working/peft-EEG-training-1735478979/checkpoint-600/scheduler.pt  
  inflating: /content/checkpoint-600/kaggle/working/peft-EEG-trai

In [ ]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 require

In [ ]:
!pip install gdown

import gdown
file_id = '1Y2cTYR_t_10NAbznspE5bBjuATPdTgtq'
gdown.download(f'https://drive.google.com/uc?export=download&id={file_id}', 'file.zip', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1Y2cTYR_t_10NAbznspE5bBjuATPdTgtq
From (redirected): https://drive.google.com/uc?export=download&id=1Y2cTYR_t_10NAbznspE5bBjuATPdTgtq&confirm=t&uuid=61d22510-1d69-4634-9bec-71b37207e129
To: /content/file.zip
100%|██████████| 134M/134M [00:00<00:00, 154MB/s]


'file.zip'

In [ ]:
import zipfile
import os

zip_path = '/kaggle/working/file.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    extract_path = 'extracted_files'
    zip_ref.extractall(extract_path)

extracted_files = os.listdir(extract_path)
print(f"Extracted files and directories: {extracted_files}")

Extracted files and directories: ['AISContest_Data']


In [ ]:
import os
import numpy as np

data_path = '/kaggle/working/extracted_files/AISContest_Data'
data_dict = {}

for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)

    if os.path.isdir(folder_path):
        if folder == '0':
            label = "Drowsy Wakefulness"
        elif folder =='1':
            label = "Light Sleep Onset"
        elif folder == '2':
            label = "Stable Light Sleep"
        elif folder == '3':
            label = "Deep Restorative Sleep"
        else:
            label = "Dreaming REM Sleep"

        # label = int(folder)
        files = []

        for file in os.listdir(folder_path):
            if file.endswith('.npy'):
                file_path = os.path.join(folder_path, file)
                data = np.load(file_path)
                files.append(data)

        data_dict[label] = files

In [ ]:
#Number of data samples
print(len(data_dict["Drowsy Wakefulness"])+len(data_dict["Light Sleep Onset"])+len(data_dict["Stable Light Sleep"])+len(data_dict["Deep Restorative Sleep"])+len(data_dict["Dreaming REM Sleep"]))

5813


# EEG FEATURE EXTRACTION

In [ ]:
!pip install -q PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 47.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
from scipy.stats import kurtosis, skew, entropy
from scipy.signal import welch
import pywt


def calculate_time_features(data):
    mean_amp = np.mean(data, axis=1)
    rms = np.sqrt(np.mean(data ** 2, axis=1))
    variance = np.var(data, axis=1)
    return mean_amp, rms, variance


def calculate_hjorth_parameters(data):
    def hjorth_params(channel):
        diff1 = np.diff(channel)
        activity = np.var(channel)
        mobility = np.sqrt(np.var(diff1) / activity)
        complexity = np.sqrt(np.var(np.diff(diff1)) / np.var(diff1))
        return mobility, activity, complexity

    return np.apply_along_axis(hjorth_params, axis=1, arr=data)


def calculate_skewness_kurtosis(data):
    kurtosis_values = np.apply_along_axis(kurtosis, axis=1, arr=data)
    skewness_values = np.apply_along_axis(skew, axis=1, arr=data)
    return kurtosis_values, skewness_values


def calculate_frequency_features(data, sfreq, fmin=1, fmax=40):
    n_samples = data.shape[1]
    nperseg = min(128, n_samples)

    psd, freqs = welch(data, fs=sfreq, nperseg=nperseg, axis=1)
    peak_freq = freqs[np.argmax(psd, axis=0 if psd.ndim == 1 else 1)]

    # Check if psd is 1-dimensional and handle accordingly
    if psd.ndim == 1:
        spectral_entropy = entropy(psd)
        spectral_centroid = np.sum(freqs * psd) / np.sum(psd)
        spectral_flatness = np.exp(np.mean(np.log(psd))) / np.mean(psd)
    else:
        spectral_entropy = np.apply_along_axis(entropy, axis=1, arr=psd)
        spectral_centroid = np.sum(freqs * psd, axis=1) / np.sum(psd, axis=1)
        spectral_flatness = np.exp(np.mean(np.log(psd), axis=1)) / np.mean(psd, axis=1)

    freq_bands = {
        'delta': [0.5, 4],
        'theta': [4, 8],
        'alpha': [8, 12],
        'beta': [12, 30],
        'gamma': [30, 40]
    }
    band_power = {}
    for band, (low, high) in freq_bands.items():
        band_indices = np.where((freqs >= low) & (freqs <= high))[0]
        # Handle 1-dimensional psd for band power calculation
        if psd.ndim == 1:
            band_power[band] = np.sum(psd[band_indices])
        else:
            band_power[band] = np.sum(psd[:, band_indices], axis=1)

    return psd, freqs, peak_freq, spectral_entropy, spectral_centroid, spectral_flatness, band_power


def calculate_wavelet_features(data):
    wavelet = 'db4'
    max_level = pywt.dwt_max_level(data.shape[1], pywt.Wavelet(wavelet).dec_len)
    level = min(5, max_level)

    coeffs = pywt.wavedec(data, wavelet, level=level, axis=1)

    wavelet_entropies = []
    for coef in coeffs:
        norm_coef = np.abs(coef) ** 2
        norm_coef /= np.sum(norm_coef, axis=1, keepdims=True)
        wavelet_entropies.append(-np.sum(norm_coef * np.log2(norm_coef), axis=1))
    return wavelet_entropies


def process_eeg_data(data, sfreq):
    data = np.squeeze(data)
    time_features = calculate_time_features(data)
    hjorth_params = calculate_hjorth_parameters(data)
    kurtosis_values, skewness_values = calculate_skewness_kurtosis(data)
    freq_features = calculate_frequency_features(data, sfreq)
    wavelet_entropies = calculate_wavelet_features(data)

    return time_features, hjorth_params, kurtosis_values, skewness_values, freq_features, wavelet_entropies



In [ ]:
sfreq = 100
features_dict = {}


for label, files in data_dict.items():
    label_features = []
    for data in files:

        time_features, hjorth_params, kurtosis_values, skewness_values, freq_features, wavelet_entropies = process_eeg_data(data, sfreq)

        sample_features = {
            'hjorth_params': hjorth_params,
            'kurtosis_values': kurtosis_values,
            'wavelet_entropies': wavelet_entropies
        }

        label_features.append(sample_features)

    features_dict[label] = label_features


print("Extracted features for each class:")
for label, features in features_dict.items():
    print(f"{label}:")
    for feature_name, feature_values in features[0].items():
        print(f"  {feature_name}: {len(feature_values)} samples processed.")

<ipython-input-11-95c8563f8dd2>:42: RuntimeWarning: divide by zero encountered in log
  spectral_flatness = np.exp(np.mean(np.log(psd))) / np.mean(psd)


Extracted features for each class:
Deep Restorative Sleep:
  hjorth_params: 51 samples processed.
  kurtosis_values: 51 samples processed.
  wavelet_entropies: 4 samples processed.
Dreaming REM Sleep:
  hjorth_params: 51 samples processed.
  kurtosis_values: 51 samples processed.
  wavelet_entropies: 4 samples processed.
Light Sleep Onset:
  hjorth_params: 51 samples processed.
  kurtosis_values: 51 samples processed.
  wavelet_entropies: 4 samples processed.
Stable Light Sleep:
  hjorth_params: 51 samples processed.
  kurtosis_values: 51 samples processed.
  wavelet_entropies: 4 samples processed.
Drowsy Wakefulness:
  hjorth_params: 51 samples processed.
  kurtosis_values: 51 samples processed.
  wavelet_entropies: 4 samples processed.


# Saving the dataset in dictionary format

In [ ]:
new_features_dict = []

for key, values in features_dict.items():
    for value in values:
        new_entry = {
            'x': value,
            'y': key
        }
        new_features_dict.append(new_entry)

In [ ]:
#Number of data samples
len(new_features_dict)

5813

In [ ]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
import random

# Reordering the list randomly
random.shuffle(new_features_dict)

x_list = [item['x'] for item in new_features_dict]
y_list = [item['y'] for item in new_features_dict]

train_ratio = 0.80
test_ratio = 0.20

x_train, x_test, y_train, y_test = train_test_split(x_list, y_list, test_size=test_ratio, random_state=42)

dataset_dict = {
    'train': {'x': x_train, 'y': y_train},
    'test': {'x': x_test, 'y': y_test}
}

dataset = {
    'train': Dataset.from_dict(dataset_dict['train']),
    'test': Dataset.from_dict(dataset_dict['test'])
}

In [ ]:
print(len(dataset['train']), len(dataset['test']) )

4650 1163


In [ ]:
data_dict.clear()
features_dict.clear()
new_features_dict.clear()

# Install required libraries

In [ ]:
!pip install  -q -U huggingface-hub

In [ ]:
from huggingface_hub import login

#Access token for Hugging Face
login("Token Hugging Face")

print("Successfully logged in to HuggingFace!")

Successfully logged in to HuggingFace!


# Loading the required libraries

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
#from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np


# deactivate Weights and Biases during the fine-tuning process

In [ ]:
import os
# disable Weights and Biases
os.environ['WANDB_DISABLED']="true"

# Create Bitsandbytes configuration

In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

# Loading the Pre-Trained model

In [ ]:
!git config --global credential.helper store

In [ ]:
from huggingface_hub import interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) Y


In [ ]:
model_name="unsloth/Llama-3.2-3B-bnb-4bit"
device_map = {"": 0}
original_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

# Tokenization

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

# Test the Model with Zero Shot Inferencing

In [ ]:
!pip install -q pynvml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.5 MB/s eta 0:00:00


In [ ]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

In [ ]:
print_gpu_utilization()

GPU memory occupied: 2520 MB.


In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_name, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen(model,p, maxlen=100, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.1,num_beams=1,top_p=0.95,).to('cpu')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

In [ ]:
%%time
from transformers import set_seed
seed = 42
set_seed(seed)

CPU times: user 2.9 ms, sys: 972 µs, total: 3.87 ms
Wall time: 5.2 ms


In [ ]:
dataset['test'][10]['y']

'Stable Light Sleep'

In [ ]:
index = 100

prompt = dataset['test'][index]['x']


formatted_prompt = f"""Instruct: An advanced sleep stage classifier designed for analyzing EEG data.
Classify the input into one of the following sleep stage labels:
1. Drowsy Wakefulness
2. Light Sleep Onset
3. Stable Light Sleep
4. Deep Restorative Sleep
5. Dreaming REM Sleep
input : \n {prompt}\n
The output clearly classifies the input into one of the sleep stage classes (ranging from 1 to 5)
and provides the probability (in percentages) of the input belonging to each class.
Output:\n"""

res = gen(original_model,formatted_prompt,150,)
#print(res[0])
output = res[0].split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: An advanced sleep stage classifier designed for analyzing EEG data. 
Classify the input into one of the following sleep stage labels:
1. Drowsy Wakefulness 
2. Light Sleep Onset 
3. Stable Light Sleep 
4. Deep Restorative Sleep 
5. Dreaming REM Sleep
input : 
 {'hjorth_params': [[1.4803928522422247, 1.7127980775469392e-10, 1.7977954990428815], [1.1244233962300274, 4.657422119385476e-11, 1.6108636110220471], [1.4173455319959478, 3.4153236569396347e-12, 1.7502717390784688], [1.4608436452782207, 1.5029765701337142e-12, 1.8100906292509857], [1.431546985114563, 1.1301128355443903e-12, 1.7695850176929535], [1.5218811488519972, 9.422468294205915e-13, 1.8320771129524638], [1.6272138477906635, 6.719777317038754e-13, 1.8923017399873017], [1.7175123589408614, 3.351197195811809e-13, 1.9120458643254787], [1.4970298891170855, 1.4488139773919223e-13, 1.828950951188861], [1.44297

# Pre-processing dataset

In [ ]:
def create_prompt_formats(sample):
    """
    Formats a concise and effective prompt for fine-tuning the Phi-3 model
    on EEG-based sleep stage classification.
    """

    # Description of the task and sleep stage labels
    INTRO_BLURB = (
        "Classify the input into one of the following sleep stage labels:\n"
        "1. Drowsy Wakefulness\n"
        "2. Light Sleep Onset\n"
        "3. Stable Light Sleep\n"
        "4. Deep Restorative Sleep\n"
        "5. Dreaming REM Sleep\n\n"
    )

    # Clear instruction for the model
    INSTRUCTION_KEY = (
        "### Instruction:\n"
        "An advanced sleep stage classifier designed for analyzing EEG data."
    )

    # Response format for consistency
    RESPONSE_KEY = "### Sleep Stage Label:"

    # Constraints and expected output format
    END_KEY = (
        "The output clearly classifies the input into one of the sleep stage classes (ranging from 1 to 5) and provides the probability (in percentages) of the input belonging to each class.\nOutput:\n"
    )

    # Construct the full prompt
    instruction = f"{INSTRUCTION_KEY}\nInput:\n{sample['x']}\n"
    response = f"{RESPONSE_KEY} {sample['y']}"
    end = f"{END_KEY}"

    # Combine all sections to create the final prompt
    parts = [INTRO_BLURB, instruction, response, end]
    formatted_prompt = "\n\n".join(parts)

    # Update the sample dictionary with the formatted prompt
    sample["text"] = formatted_prompt

    return sample


In [ ]:
from functools import partial

def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length




def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )


# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int,seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """

    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)#, batched=True)

    # Apply preprocessing to each batch of the dataset & and remove 'instruction', 'context', 'response', 'category' fields
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=['x', 'y'],
    )

    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset

In [ ]:
## Pre-process dataset
max_length = get_max_length(original_model)
print(max_length)

train_dataset = preprocess_dataset(tokenizer, max_length,seed, dataset['train'])

Found max lenth: 131072
131072
Preprocessing dataset...


Map:   0%|          | 0/4650 [00:00<?, ? examples/s]

Map:   0%|          | 0/4650 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4650 [00:00<?, ? examples/s]

# Preparing the model for QLoRA

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 394177536
all model parameters: 1803463680
percentage of trainable model parameters: 21.86%


In [ ]:
print(original_model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

config = LoraConfig(
    r=4,  # Rank
    lora_alpha=4,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # Attention layers in the Llama model
        "gate_proj", "up_proj", "down_proj"      # MLP layers in the Llama model
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

# 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
original_model.gradient_checkpointing_enable()

# 2 - Using the prepare_model_for_kbit_training method from PEFT
original_model = prepare_model_for_kbit_training(original_model)

peft_model = get_peft_model(original_model, config)

# Setup PEFT for Fine-Tuning

In [ ]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 6078464
all model parameters: 1809542144
percentage of trainable model parameters: 0.34%


# Train PEFT Adapter

In [ ]:
output_dir = f'./peft-EEG-training-{str(int(time.time()))}'
import transformers
from transformers import Trainer, TrainingArguments

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    max_steps=600,
    learning_rate=2e-4,
    optim="adamw_8bit",
    logging_steps=100, #1
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=100,
    do_eval=False,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir='True',
    group_by_length=True,
    fp16=True
)


peft_model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [ ]:
peft_training_args.device

device(type='cuda', index=0)

In [ ]:
peft_trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
100,3.567800
200,3.544900
300,3.540000
400,3.542500
500,3.539700
600,3.538800


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/lib/python3.10/site-pac

TrainOutput(global_step=600, training_loss=3.5456341044108073, metrics={'train_runtime': 23019.846, 'train_samples_per_second': 0.052, 'train_steps_per_second': 0.026, 'total_flos': 8.560415550121574e+16, 'train_loss': 3.5456341044108073, 'epoch': 0.25806451612903225})

In [ ]:
!zip -r checkpoint-600.zip /kaggle/working/peft-EEG-training-1735478979/checkpoint-600

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/peft-EEG-training-1735478979/checkpoint-600/ (stored 0%)
  adding: kaggle/working/peft-EEG-training-1735478979/checkpoint-600/adapter_config.json (deflated 56%)
  adding: kaggle/working/peft-EEG-training-1735478979/checkpoint-600/training_args.bin (deflated 52%)
  adding: kaggle/working/peft-EEG-training-1735478979/checkpoint-600/rng_state.pth (deflated 25%)
  adding: kaggle/working/peft-EEG-training-1735478979/checkpoint-600/optimizer.pt (deflated 11%)
  adding: kaggle/working/peft-EEG-training-1735478979/checkpoint-600/adapter_model.safetensors (deflated 8%)
  adding: kaggle/working/peft-EEG-training-1735478979/checkpoint-600/README.md (deflated 66%)
  adding: kaggle/working/peft-EEG-training-1735478979/checkpoint-600/scheduler.pt (deflated 56%)
  adding: kaggle/working/peft-EEG-training-1735478979/checkpoint-600/trainer_state.json (deflated 65%)


In [ ]:
print_gpu_utilization()

GPU memory occupied: 14172 MB.


In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Free memory for merging weights
del original_model
del peft_trainer
torch.cuda.empty_cache()

In [ ]:
print_gpu_utilization()

GPU memory occupied: 3412 MB.


# Evaluate the Model Qualitatively (Human Evaluation)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id=model_name
base_model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

In [ ]:
from peft import PeftModel

#ft_model = PeftModel.from_pretrained(base_model, "/kaggle/working/peft-EEG-training-1735478979/checkpoint-600",torch_dtype=torch.float16,is_trainable=False)
ft_model = PeftModel.from_pretrained(base_model, "/content/checkpoint-600/kaggle/working/peft-EEG-training-1735478979/checkpoint-600",torch_dtype=torch.float16,is_trainable=False)

# Evaluate the Model Qualitatively (Human Evaluation)

In [ ]:
%%time
from transformers import set_seed
set_seed(seed)

index = 2
x = dataset['test'][index]['x']
y = dataset['test'][index]['y']

prompt = f""""Instruct: An advanced sleep stage classifier designed for analyzing EEG data.
Classify the input into one of the following sleep stage labels:
1. Drowsy Wakefulness
2. Light Sleep Onset
3. Stable Light Sleep
4. Deep Restorative Sleep
5. Dreaming REM Sleep
input : \n {x}\n
(The output is Classify sleep stage label for input.)
Output:\n"""


peft_model_res = gen(ft_model,prompt,100,)
peft_model_output = peft_model_res[0].split('Output:\n')[1]
#print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN Stages sleep number:\n{y}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
"Instruct: An advanced sleep stage classifier designed for analyzing EEG data. 
Classify the input into one of the following sleep stage labels:
1. Drowsy Wakefulness  
2. Light Sleep Onset  
3. Stable Light Sleep  
4. Deep Restorative Sleep  
5. Dreaming REM Sleep  
input : 
 {'hjorth_params': [[0.9518534588439543, 7.600260869921925e-11, 1.55328263138652], [0.7662341391997871, 8.117545471800677e-11, 1.2058633136202694], [0.7304528928773373, 1.3186699837014625e-11, 1.1750514849080411], [1.0318495611334475, 1.8179743509413643e-12, 1.5642256851076155], [1.1408323815430896, 6.141499487014688e-13, 1.5730730878119499], [1.2813090965651144, 2.7112438900438205e-13, 1.7795079811059966], [1.415752603045438, 1.5186544364590866e-13, 1.753262664901251], [1.3378773184919677, 1.5344377342072014e-13, 1.65726509610303], [1.3353596029490096, 1.5843596992320144e-13, 1.6416248920912868], [1.2

# Evaluate the Model Quantitatively

In [ ]:
original_model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [ ]:
import pandas as pd

x = dataset['test']['x']
human_baseline_y = dataset['test']['y']

original_model_y = []
instruct_model_y = []
peft_model_y = []

for idx, x_i in enumerate(x):
    print("test data number : ", idx+1)
    human_baseline_text_output = human_baseline_y[idx]

    prompt = f""""Instruct: An advanced sleep stage classifier designed for analyzing EEG data.
    Classify the input into one of the following sleep stage labels:
    1. Drowsy Wakefulness
    2. Light Sleep Onset
    3. Stable Light Sleep
    4. Deep Restorative Sleep
    5. Dreaming REM Sleep
    input : \n {x_i}\n
    (The output is the sleep stage label for the input.)
    Output:\n"""


    original_model_res = gen(original_model,prompt,100,)
    original_model_text_output = original_model_res[0].split('Output:\n')[1]

    peft_model_res = gen(ft_model,prompt,100,)
    peft_model_output = peft_model_res[0].split('Output:\n')[1]
    print(peft_model_output)
    peft_model_text_output, success, result = peft_model_output.partition('###')

    original_model_y.append(original_model_text_output)
    peft_model_y.append(peft_model_text_output)

zipped_y = list(zip(human_baseline_y, original_model_y, peft_model_y))

df = pd.DataFrame(zipped_y, columns = ['human_baseline_y', 'original_model_y', 'peft_model_y'])
df

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


test data number :  1


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


    5
    """
    output = {
        'hjorth_params': [[1.408144840152645, 1.156272405203362e-12, 1.677576970142494], [1.386693594574586, 1.027619577752355e-12, 1.688542767297291], [1.430578979549915, 5.955156206216666e-13, 1
test data number :  2


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


    3
    """
    output = {
        'hjorth_params': [[0.6616137828726207, 1.2405154948467125e-09, 1.586639359472557], [0.6164237058639771, 5.94786612103243e-10, 1.391294037344969], [0.9272157643296495, 1.4829006247076462e
test data number :  405


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


    4
    """
    output = {
        'hjorth_params': [[1.359012952291339, 1.230090549252464e-10, 1.684672957757643], [1.141272573518808, 1.043760614330123e-10, 1.628551144378128], [1.210108871393763, 1.002140446849114e-11, 1
test data number :  406


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
